In [ ]:
!pip install sentence-transformers faiss-cpu transformers peft bitsandbytes accelerate langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
# 假設有一篇長文章
long_text = "這是一篇很長的醫療文件內容..." * 100

# 將其切段（chunking）
def chunk_text(text, max_length=300):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

chunks = chunk_text(long_text)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

# 建立向量索引資料庫（FAISS）
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# 儲存段落對應
id_to_text = {i: para for i, para in enumerate(chunks)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 使用者問題
question = "我牙齒敏感怎麼辦？"
query_vector = model.encode([question]).astype("float32")

# 查詢最相似的段落 Top-K
D, I = index.search(query_vector, k=3)
retrieved_text = "\n".join([id_to_text[i] for i in I[0]])

In [ ]:
from transformers import pipeline

# 建立 Prompt
prompt = f"""你是一位牙科助理。請根據以下資料回答問題：
{retrieved_text}
問題：{question}
"""

# 使用生成模型（TinyLlama 或 ChatGLM）
generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
result = generator(prompt, max_new_tokens=200)
print(result[0]['generated_text'])

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


你是一位牙科助理。請根據以下資料回答問題：
容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...
這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內
這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內容...這是一篇很長的醫療文件內
問題：我牙齒敏感怎麼辦？

回答：根據以下資料，你的牙齒敏感程度可以通過以下方式辦：

1. 試試：首先，你可以試試牙齒的敏感程度。試試的方法是，你可以把牙齒放在一個紅色的紙上，然後把紅色的紙擺在你的腦前面。如果你的牙齒敏感，那麼你可以把紅色的紙擺在你的腦前面，��


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig

# 載入模型（無量化、不使用 GPU）
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# 建立 LoRA 設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # 注意：依模型架構調整
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# 將模型包裝為 LoRA 可微調版本
model = get_peft_model(model, peft_config)

# 顯示可訓練參數（確認微調啟用）
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [ ]:
TTFT = 0.5  # First Token Time（秒）
TPOT = 0.02  # 每個 token 的產生時間（秒）
response_token_count = 100

Latency = TTFT + (TPOT * response_token_count)
print(f"推論延遲：{Latency:.2f} 秒")

推論延遲：2.50 秒


In [ ]:
prompt_template = """你是一位助理醫師。根據以下資料回答問題：
{context}
使用者提問：{question}
"""

In [ ]:
# 預先準備：model（SentenceTransformer）、index、id_to_text、LLM（如 pipeline）

from transformers import pipeline
qa_generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

while True:
    user_input = input("👤 問題（輸入 q 離開）：")
    if user_input.lower() == "q":
        break

    # 1. 問題轉向量
    query_vector = model.encode([user_input]).astype("float32")

    # 2. 檢索語意最接近段落
    D, I = index.search(query_vector, k=3)
    retrieved = "\n".join([id_to_text[i] for i in I[0]])

    # 3. 建立 Prompt
    prompt_template = """你是一位助理醫師。根據以下資料回答問題：
{context}
使用者提問：{question}
"""
    prompt = prompt_template.format(context=retrieved, question=user_input)

    # 4. 使用 LLM 生成回答
    result = qa_generator(prompt, max_new_tokens=150)
    print("🤖 回答：", result[0]["generated_text"])

In [ ]:
!pip install sentence-transformers faiss-cpu transformers peft accelerate

In [ ]:
def chunk_text(text, max_length=300):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

text = "這是很長的醫療資料..." * 100
chunks = chunk_text(text)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

index = faiss.IndexFlatL2(384)
index.add(embeddings.astype("float32"))
id_to_text = {i: chunk for i, chunk in enumerate(chunks)}

In [ ]:
question = "牙齒敏感怎麼處理？"
q_vector = model.encode([question]).astype("float32")
D, I = index.search(q_vector, k=3)
retrieved = "\n".join([id_to_text[i] for i in I[0]])

In [ ]:
from transformers import pipeline
qa_generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

prompt_template = """我是一位助理醫師。根據以下資料回答問題：
{context}
使用者提問：{question}"""

prompt = prompt_template.format(context=retrieved, question=question)
result = qa_generator(prompt, max_new_tokens=150)
print(result[0]['generated_text'])

Device set to use cpu


我是一位助理醫師。根據以下資料回答問題：
這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...
這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...
這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...
使用者提問：牙齒敏感怎麼處理？
這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...這是很長的醫療資料...

In [ ]:
from peft import get_peft_model, LoraConfig
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

peft_config = LoraConfig(
    r=8, lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [ ]:
TTFT = 0.5  # 首 token 出現時間
TPOT = 0.02  # 每個 token 平均生成時間
response_token_count = 100
Latency = TTFT + (TPOT * response_token_count)
print(f"推論延遲：{Latency:.2f} 秒")  # ➜ 2.50 秒

推論延遲：2.50 秒


In [ ]:
# 安裝必要套件（若尚未安裝）
# pip install sentence-transformers faiss-cpu transformers

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

# ✅ Step 1: 建立知識段落（可以改成你的資料）
chunks = [
    "牙齒敏感可能與琺瑯質受損有關，可使用抗敏感牙膏。",
    "刷牙時出血可能是牙周病的前兆，建議就診檢查牙齦狀況。",
    "進食冷熱食物疼痛可能與牙髓炎或裂齒有關，建議及早處理。",
]

# ✅ Step 2: 建立嵌入模型並轉向量
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks)

# ✅ Step 3: 建立 FAISS 向量索引
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# ✅ Step 4: 段落對應表（用於查詢後還原文字）
id_to_text = {i: chunk for i, chunk in enumerate(chunks)}

# ✅ Step 5: 載入 LLM 回答模型（純 CPU）
qa_generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# ✅ Step 6: 終端機互動回圈
prompt_template = """我是一位助理醫師。根據以下資料回答問題：
{context}
使用者提問：{question}"""

print("🤖 RAG 醫療助手已啟動，輸入問題即可開始對話！（輸入 q 離開）\n")

while True:
    user_input = input("👤 請輸入問題：")
    if user_input.lower() == "q":
        print("👋 已結束對話，謝謝使用！")
        break

    # ➤ 查詢語意相近段落
    query_vector = embedding_model.encode([user_input]).astype("float32")
    D, I = index.search(query_vector, k=3)
    retrieved = "\n".join([id_to_text[i] for i in I[0]])

    # ➤ 合成 Prompt 並回答
    prompt = prompt_template.format(context=retrieved, question=user_input)
    result = qa_generator(prompt, max_new_tokens=150)
    print("\n🤖 回答：", result[0]["generated_text"])
    print("-" * 50)


Device set to use cpu


🤖 RAG 醫療助手已啟動，輸入問題即可開始對話！（輸入 q 離開）

👤 請輸入問題：牙齒敏感

🤖 回答： 我是一位助理醫師。根據以下資料回答問題：
牙齒敏感可能與琺瑯質受損有關，可使用抗敏感牙膏。
刷牙時出血可能是牙周病的前兆，建議就診檢查牙齦狀況。
進食冷熱食物疼痛可能與牙髓炎或裂齒有關，建議及早處理。
使用者提問：牙齒敏感可能與琺瑯質受損有關，可使用抗敏感牙膏。
答：牙齒敏感可能與琺瑯質受損有關，抗敏感牙膏可以協助防止牙齒敏感。

詳細資料：
牙齒敏感
--------------------------------------------------
👤 請輸入問題：q
👋 已結束對話，謝謝使用！
